In [260]:
from requests import request
from bs4 import BeautifulSoup as soup
from pprint import pprint
import re
import libtorrent
import time
import sys
import os.path
import difflib 
from collections import OrderedDict
import itertools


def get_latest_movirulz_url(keyword, n_results=1):
    headers = {"User-Agent": "Mozilla/5.0"}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    query = keyword
    query = urllib.parse.quote_plus(query)
    number_result = n_results
    google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
    response = requests.get(google_url, headers=headers, cookies=cookies)
    soup = BeautifulSoup(response.text, "html.parser")
    result = soup.find_all('div')
    results=[re.search('\/url\?q\=(.*)\&sa',str(i.find('a', href = True)['href'])) for i in result if "url" in str(i)]
    links=[i.group(1) for i in results if i != None]
    return links[0]

movierulz_url = get_latest_movirulz_url('movierulz')+'?s='

movie = input("Enter Movie Name:")
movie = movie.replace(" ","%20")

url = movierulz_url+movie

response = request('GET',url)


Enter Movie Name:vikram vedha


In [261]:
html_page_soup_1 = soup(response.text, "html.parser")
search_container = html_page_soup_1.findAll("div", {"class": "content home_style"})
try:
    movie_titles =  search_container[0].findAll("b")
except:
    print("could'nt find movie")
movie_title_ls = [] ; trimmed_movie_title_ls = []

rep = {"Full": "", "Movie": "","Watch":" ","Online":"","Free":""}
rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))

for i in movie_titles:
    movie_title_ls.append(i.text.strip())
    trimmed_movie_name = pattern.sub(lambda m: rep[re.escape(m.group(0))],i.text.strip())
    trimmed_movie_title_ls.append(trimmed_movie_name.strip())
    
if len(trimmed_movie_title_ls)==1:
    best_match = trimmed_movie_title_ls[0]
else:
    best_match = difflib.get_close_matches(movie,trimmed_movie_title_ls,n=1,cutoff=0.2)[0]

close_results=dict()
for _ in trimmed_movie_title_ls:
    score = difflib.SequenceMatcher(None, _, best_match).ratio()
    close_results.update({score:_})
close_results = OrderedDict(sorted(close_results.items(),reverse=True))   
close_results = dict(itertools.islice(close_results.items(), 5))
print(close_results)

print(trimmed_movie_title_ls)

get_torrent(trimmed_movie_title_ls[0])

{1.0: 'Vikram Vedha (2018) HDRip Hindi Dubbed'}
['Vikram Vedha (2018) HDRip Hindi Dubbed']
Vikram Vedha (2018) HDRip Hindi Dubbed Movie Watch Online Free 

[('1 gb', '720p')]
{'1 gb': 'magnet:?xt=urn:btih:1E808A4C817B7C2EDB402D9374A94C140EC4CFDD&dn=www.4movierulz.us%20-%20Vikram%20Vedha%20%282018%29%20720p%20Hindi%20HD%20AVC%20AAC%201%20GB.mp4&tr=udp%3a%2f%2ftracker.leechers-paradise.org%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.opentrackr.org%3a1337%2fannounce&tr=udp%3a%2f%2feddie4.nl%3a6969%2fannounce&tr=udp%3a%2f%2f9.rarbg.me%3a2730%2fannounce&tr=udp%3a%2f%2f9.rarbg.to%3a2710%2fannounce&tr=udp%3a%2f%2fpublic.popcorn-tracker.org%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.opentrackr.org%3a1337%2fannounce&tr=udp%3a%2f%2fp4p.arenabg.ch%3a1337%2fannounce&tr=udp%3a%2f%2ftracker.internetwarriors.net%3a1337%2fannounce&tr=udp%3a%2f%2ftracker.coppersurfer.tk%3a6969%2fannounce&tr=http%3a%2f%2ftorrentsmd.com%3a8080%2fannounce'}


In [ ]:
def mag2tor(name,mag_link):
    sess = libtorrent.session()
    prms = {
        'save_path':os.path.abspath(os.path.curdir),
    }
    torr = libtorrent.add_magnet_uri(sess, mag_link, prms)
    dots = 0
    while not torr.has_metadata():
        dots += 1
        sys.stdout.write('.')
        sys.stdout.flush()
    if (dots): sys.stdout.write('\n')
    sess.pause()
    tinf = torr.get_torrent_info()
    f = open(name + '.torrent', 'wb')
    f.write(libtorrent.bencode(
        libtorrent.create_torrent(tinf).generate()))
    f.close()


In [256]:
def get_torrent(movie_name):
    
    movie = movie_name.replace(" ","-")
    base_url= "https://ww5.7movierulz.tc/"
    url = base_url+movie
    response = request('GET',url)
    
    html_page_soup = soup(response.text, "html.parser")
    container = html_page_soup.findAll("a", {"class": "mv_button_css"})
    movie_title = html_page_soup.findAll("h2", {"class": "entry-title"})

    torrent_quality = html_page_soup.findAll("small")
    movie_title = movie_title[0].text.strip()
    print(movie_title,"\n")

    torrent_size_ls = [] ; torrent_resolution_ls = [] ; magnet_link_ls = [] ; magnets = dict()
    for i,torrent in enumerate(torrent_quality):
        if i%2==0:
            torrent_size_ls.append(torrent.text.strip())
        else:
            torrent_resolution_ls.append(torrent.text.strip())

    torrent_info = dict(zip(torrent_size_ls,torrent_resolution_ls))
    
    keyorder = ['8K','4K','4K SDR','1080p','1440p','720p', '480p', '320p','240p','144p']
    torrent_info = sorted(torrent_info.items(), key=lambda i:keyorder.index(i[1]))
    
    pprint(torrent_info)
    for link in container:
        magnet_link_ls.append(link.get('href'))

    for i,j in zip(magnet_link_ls,torrent_info):
        magnets.update({j[0]:i})

    #pprint(magnets)
    #mag2tor(trimmed_movie_title_ls[0],magnets['700 mb'])
    
